In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 32
shuffle = True
seed = None

input_data_folder = "Data"
training_data_file = "Training-datasets-PredNTS.txt"
independent_data_file = "independent dataset-PredNTS.txt"

iNitroY_input_data_folder = "Data\\iNitroY-Deep-Dataset"
iNitroY_pos_data_file = "raw-nitrotyrosine-pos.fasta"
iNitroY_neg_data_file = "cdhit70-nitrotyr-neg.fasta"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

from Bio import SeqIO

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 10, kernel_length_1 = 5, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
                 lstm_decode_units = 10, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 10,  kernel_length_2 = 3, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
                 dense_decode_units = 64, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = None):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(x4)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss)

    return model

In [8]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 10)       1060        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 10)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 13, 10)       0           ['activation[0][0]']             
                                                                                              

# Train data preparation

In [9]:
def read_fasta_file(file_path):
    
    openFile = open(file_path)
    fastaSequences = SeqIO.parse(openFile, "fasta")

    name_list = []
    seq_list = []

    for fasta in fastaSequences: 
        name_list.append(fasta.id)
        seq_list.append(str(fasta.seq))

    openFile.close()
    
    return name_list, seq_list

In [10]:
##################################################################################
##### read PredNTS training file
##################################################################################
train_file_path = os.path.join(input_data_folder, training_data_file)
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
train_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
train_data.head()

train_seq_list = list(train_data['Sequence'])
train_seq_label = [1 if val == 1 else 0 
                   for val in train_data["label_original"]]

##################################################################################
##### read iNitroY file
##################################################################################

iNitroY_pos_file_path = os.path.join(iNitroY_input_data_folder, iNitroY_pos_data_file)
_, iNitroY_pos_seq_list = read_fasta_file(iNitroY_pos_file_path)

iNitroY_neg_file_path = os.path.join(iNitroY_input_data_folder, iNitroY_neg_data_file)
_, iNitroY_neg_seq_list = read_fasta_file(iNitroY_neg_file_path)

iNitroY_pos_seq_list = [val.replace('X', '-') for val in iNitroY_pos_seq_list]
iNitroY_neg_seq_list = [val.replace('X', '-') for val in iNitroY_neg_seq_list]

# remove duplicates in data
iNitroY_pos_seq_list = list(set(iNitroY_pos_seq_list))
iNitroY_neg_seq_list = list(set(iNitroY_neg_seq_list))

iNitroY_seq_list = iNitroY_pos_seq_list + iNitroY_neg_seq_list

iNitroY_seq_label_list = ([1] * len(iNitroY_pos_seq_list)) + ([0] * len(iNitroY_neg_seq_list))

##################################################################################
##### merge both for training
##################################################################################

seq_list = train_seq_list + iNitroY_seq_list
label_list = train_seq_label + iNitroY_seq_label_list

all_train_data = pd.DataFrame({'Sequence':seq_list, 'label':label_list})
all_train_data = all_train_data.drop_duplicates().reset_index(drop=True)

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in all_train_data['Sequence']]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i

##################################################################################
##### Create OHE of sequence
##################################################################################
all_train_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                            for val in all_train_data["Sequence"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

features = np.array(list(all_train_data['OHE_Sequence']))
labels = np.array(list(all_train_data['label']))
labels = labels.reshape((labels.shape[0], 1))

input_seq_shape = features[0].shape

folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Training

In [11]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/100
74/74 [==============================] - ETA: 0s - loss: 0.8301
Epoch 1: val_loss improved from inf to 0.80717, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [==============================] - 5s 15ms/step - loss: 0.8301 - val_loss: 0.8072
Epoch 2/100
74/74 [==============================] - ETA: 0s - loss: 0.8100
Epoch 2: val_loss improved from 0.80717 to 0.79460, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.8100 - val_loss: 0.7946
Epoch 3/100
73/74 [============================>.] - ETA: 0s - loss: 0.7967
Epoch 3: val_loss improved from 0.79460 to 0.78396, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.7969 - val_loss: 0.784

74/74 [==============================] - ETA: 0s - loss: 0.6043
Epoch 26: val_loss did not improve from 0.63036
74/74 [==============================] - 1s 8ms/step - loss: 0.6043 - val_loss: 0.6309
Epoch 27/100
73/74 [============================>.] - ETA: 0s - loss: 0.6028
Epoch 27: val_loss improved from 0.63036 to 0.62693, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.6028 - val_loss: 0.6269
Epoch 28/100
72/74 [============================>.] - ETA: 0s - loss: 0.5978
Epoch 28: val_loss did not improve from 0.62693
74/74 [==============================] - 1s 8ms/step - loss: 0.5959 - val_loss: 0.6272
Epoch 29/100
73/74 [============================>.] - ETA: 0s - loss: 0.6069
Epoch 29: val_loss improved from 0.62693 to 0.62234, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [===============

74/74 [==============================] - 1s 8ms/step - loss: 0.5531 - val_loss: 0.5972
Epoch 56/100
71/74 [===========================>..] - ETA: 0s - loss: 0.5599
Epoch 56: val_loss did not improve from 0.59716
74/74 [==============================] - 1s 8ms/step - loss: 0.5590 - val_loss: 0.6002
Epoch 57/100
72/74 [============================>.] - ETA: 0s - loss: 0.5499
Epoch 57: val_loss improved from 0.59716 to 0.59500, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold0.hdf5
74/74 [==============================] - 1s 9ms/step - loss: 0.5491 - val_loss: 0.5950
Epoch 58/100
73/74 [============================>.] - ETA: 0s - loss: 0.5571
Epoch 58: val_loss did not improve from 0.59500
74/74 [==============================] - 1s 8ms/step - loss: 0.5555 - val_loss: 0.5976
Epoch 59/100
71/74 [===========================>..] - ETA: 0s - loss: 0.5415
Epoch 59: val_loss did not improve from 0.59500
74/74 [==============================

Epoch 88/100
72/74 [============================>.] - ETA: 0s - loss: 0.5089
Epoch 88: val_loss did not improve from 0.57691
74/74 [==============================] - 1s 8ms/step - loss: 0.5083 - val_loss: 0.5843
Epoch 89/100
71/74 [===========================>..] - ETA: 0s - loss: 0.5096
Epoch 89: val_loss did not improve from 0.57691
74/74 [==============================] - 1s 8ms/step - loss: 0.5103 - val_loss: 0.5802
Epoch 90/100
71/74 [===========================>..] - ETA: 0s - loss: 0.5235
Epoch 90: val_loss did not improve from 0.57691
74/74 [==============================] - 1s 8ms/step - loss: 0.5232 - val_loss: 0.5815
Epoch 91/100
73/74 [============================>.] - ETA: 0s - loss: 0.5211
Epoch 91: val_loss did not improve from 0.57691
74/74 [==============================] - 1s 8ms/step - loss: 0.5206 - val_loss: 0.5784
Epoch 92/100
69/74 [==========================>...] - ETA: 0s - loss: 0.5128
Epoch 92: val_loss did not improve from 0.57691
74/74 [====================

74/74 [==============================] - 1s 8ms/step - loss: 0.6350 - val_loss: 0.6544
Epoch 18/100
71/74 [===========================>..] - ETA: 0s - loss: 0.6384
Epoch 18: val_loss improved from 0.65440 to 0.64450, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold1.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.6386 - val_loss: 0.6445
Epoch 19/100
72/74 [============================>.] - ETA: 0s - loss: 0.6288
Epoch 19: val_loss improved from 0.64450 to 0.64219, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold1.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.6307 - val_loss: 0.6422
Epoch 20/100
73/74 [============================>.] - ETA: 0s - loss: 0.6281
Epoch 20: val_loss improved from 0.64219 to 0.63681, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold1.hdf5
74/74 [============

...

73/74 [============================>.] - ETA: 0s - loss: 0.5792
Epoch 37: val_loss improved from 0.60721 to 0.60507, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold3.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5784 - val_loss: 0.6051
Epoch 38/100
72/74 [============================>.] - ETA: 0s - loss: 0.5679
Epoch 38: val_loss did not improve from 0.60507
74/74 [==============================] - 1s 8ms/step - loss: 0.5695 - val_loss: 0.6078
Epoch 39/100
72/74 [============================>.] - ETA: 0s - loss: 0.5725
Epoch 39: val_loss did not improve from 0.60507
74/74 [==============================] - 1s 8ms/step - loss: 0.5725 - val_loss: 0.6113
Epoch 40/100
73/74 [============================>.] - ETA: 0s - loss: 0.5617
Epoch 40: val_loss improved from 0.60507 to 0.60473, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold3.hdf5
74/74 [===============

Epoch 72/100
74/74 [==============================] - ETA: 0s - loss: 0.5313
Epoch 72: val_loss did not improve from 0.59649
74/74 [==============================] - 1s 8ms/step - loss: 0.5313 - val_loss: 0.6077
Epoch 73/100
73/74 [============================>.] - ETA: 0s - loss: 0.5152
Epoch 73: val_loss did not improve from 0.59649
74/74 [==============================] - 1s 8ms/step - loss: 0.5168 - val_loss: 0.5994
Epoch 74/100
73/74 [============================>.] - ETA: 0s - loss: 0.5145
Epoch 74: val_loss did not improve from 0.59649
74/74 [==============================] - 1s 8ms/step - loss: 0.5133 - val_loss: 0.6119
Epoch 75/100
72/74 [============================>.] - ETA: 0s - loss: 0.5352
Epoch 75: val_loss improved from 0.59649 to 0.59333, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold3.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5367 - val_loss: 0.5933
Epoch 76/100
72/74 [=================

74/74 [==============================] - ETA: 0s - loss: 0.7611
Epoch 6: val_loss improved from 0.76691 to 0.75529, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.7611 - val_loss: 0.7553
Epoch 7/100
72/74 [============================>.] - ETA: 0s - loss: 0.7486
Epoch 7: val_loss improved from 0.75529 to 0.74238, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.7488 - val_loss: 0.7424
Epoch 8/100
73/74 [============================>.] - ETA: 0s - loss: 0.7430
Epoch 8: val_loss improved from 0.74238 to 0.72640, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.7434 - val_loss: 0.7264
Epoch 9/100
73/74 [==================

Epoch 33/100
71/74 [===========================>..] - ETA: 0s - loss: 0.5840
Epoch 33: val_loss did not improve from 0.58556
74/74 [==============================] - 1s 8ms/step - loss: 0.5889 - val_loss: 0.5881
Epoch 34/100
73/74 [============================>.] - ETA: 0s - loss: 0.5874
Epoch 34: val_loss improved from 0.58556 to 0.58300, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5863 - val_loss: 0.5830
Epoch 35/100
73/74 [============================>.] - ETA: 0s - loss: 0.5780
Epoch 35: val_loss improved from 0.58300 to 0.57981, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5765 - val_loss: 0.5798
Epoch 36/100
73/74 [============================>.] - ETA: 0s - loss: 0.5806
Epoch 36: val_loss did not improve from 0.57981
74/74 [==

73/74 [============================>.] - ETA: 0s - loss: 0.5493
Epoch 67: val_loss did not improve from 0.56657
74/74 [==============================] - 1s 7ms/step - loss: 0.5498 - val_loss: 0.5706
Epoch 68/100
73/74 [============================>.] - ETA: 0s - loss: 0.5461
Epoch 68: val_loss improved from 0.56657 to 0.56615, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5459 - val_loss: 0.5661
Epoch 69/100
73/74 [============================>.] - ETA: 0s - loss: 0.5336
Epoch 69: val_loss improved from 0.56615 to 0.56597, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\bestModel-fold4.hdf5
74/74 [==============================] - 1s 8ms/step - loss: 0.5355 - val_loss: 0.5660
Epoch 70/100
72/74 [============================>.] - ETA: 0s - loss: 0.5463
Epoch 70: val_loss did not improve from 0.56597
74/74 [===============

## k-fold Training evaluation

In [12]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.714332,0.665938,0.781233,0.757308,0.665938,0.424997
Train,0.819977,0.772953,0.891502,0.864983,0.772953,0.641003


In [13]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.720812,0.675277,"[0.0, 0.0, 0.0, 0.03076923076923077, 0.0307692...","[0.0, 0.0030211480362537764, 0.006042296072507...","[1.9448644, 0.9448644, 0.94219625, 0.8853597, ...",0.784941,0.759375,0.675277,0.436315
3,1,Test,0.694915,0.656250,"[0.0, 0.0038461538461538464, 0.007692307692307...","[0.0, 0.0, 0.0, 0.006060606060606061, 0.006060...","[1.9125819, 0.912582, 0.90220594, 0.88552946, ...",0.764149,0.724551,0.656250,0.380143
5,2,Test,0.700000,0.645614,"[0.0, 0.0038461538461538464, 0.015384615384615...","[0.0, 0.0, 0.0, 0.0030303030303030303, 0.00303...","[1.8855293, 0.88552934, 0.86125296, 0.8590325,...",0.779213,0.750820,0.645614,0.399024
7,3,Test,0.727119,0.681319,"[0.0, 0.0038461538461538464, 0.034615384615384...","[0.0, 0.0, 0.0, 0.0030303030303030303, 0.00303...","[1.9041855, 0.9041855, 0.8712573, 0.8622, 0.81...",0.778945,0.766562,0.681319,0.449810
9,4,Test,0.728814,0.671233,"[0.0, 0.0038461538461538464, 0.030769230769230...","[0.0, 0.0, 0.0, 0.0030303030303030303, 0.00303...","[1.9034619, 0.9034619, 0.8533277, 0.8372267, 0...",0.798916,0.785235,0.671233,0.459691


# Independent data

In [14]:
train_features = features
train_labels = labels

In [15]:
##################################################################################
##### read independent data file
##################################################################################
indpe_file_path = os.path.join(input_data_folder, independent_data_file)
indpe_data = pd.read_csv(indpe_file_path, sep='\t', header=None)
indpe_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
indpe_data.head()
    
##################################################################################
##### Create OHE of sequence
##################################################################################
indpe_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in indpe_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
indpe_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in indpe_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(list(indpe_data['OHE_Sequence']))
indpe_labels = np.array(list(indpe_data['label']))
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

input_seq_shape = indpe_features[0].shape

## Using k-fold Models

### Performance of each k-fold model

In [16]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.641469,0.252983,0.670191,0.889272,0.252983,0.186062


In [17]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.635102,0.243697,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.004892367906066536, 0.00489236790...","[1.9489883, 0.9489883, 0.8978106, 0.8920671, 0...",0.663742,0.883845,0.243697,0.167196
1,1,Independent,0.679184,0.280093,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.005870841487279843, 0.00587084148...","[1.9189126, 0.9189127, 0.88077706, 0.8686836, ...",0.688923,0.896595,0.280093,0.227045
2,2,Independent,0.643265,0.253165,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0029354207436399216, 0.0029354207...","[1.9013371, 0.90133715, 0.8670612, 0.8646005, ...",0.666259,0.889481,0.253165,0.186850
3,3,Independent,0.638367,0.248954,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9167752, 0.9167752, 0.9035742, 0.89555687, ...",0.665757,0.887550,0.248954,0.179080
4,4,Independent,0.611429,0.239006,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.00684931506849315, 0.006849315068...","[1.9023317, 0.9023317, 0.8423832, 0.83924377, ...",0.666273,0.888889,0.239006,0.170137


### Mean score with k-fold models

In [18]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.66449,0.266816,0.681557,0.892169,0.266816,0.205741


### Voting score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.651429,0.259657,0.659079,0.891963,0.259657,0.197969


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [22]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100
93/93 [==============================] - ETA: 0s - loss: 0.8230
Epoch 1: val_loss improved from inf to 0.80400, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\_fullModel.hdf5
93/93 [==============================] - 3s 17ms/step - loss: 0.8230 - val_loss: 0.8040
Epoch 2/100
89/93 [===========================>..] - ETA: 0s - loss: 0.8017
Epoch 2: val_loss improved from 0.80400 to 0.74747, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\_fullModel.hdf5
93/93 [==============================] - 1s 9ms/step - loss: 0.8024 - val_loss: 0.7475
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 0.7808
Epoch 3: val_loss improved from 0.74747 to 0.73115, saving model to Results\NT_Site_PredNTS+iNitroY_Classification_DLNN_CORENup\5fold\models\_fullModel.hdf5
93/93 [==============================] - 1s 9ms/step - loss: 0.7808 - val_loss: 0.7311
Epoch 4/100
89/93 [========================

93/93 [==============================] - 1s 9ms/step - loss: 0.5761 - val_loss: 0.5924
Epoch 36/100
90/93 [============================>.] - ETA: 0s - loss: 0.5814
Epoch 36: val_loss did not improve from 0.59239
93/93 [==============================] - 1s 8ms/step - loss: 0.5814 - val_loss: 0.6028
Epoch 37/100
86/93 [==========================>...] - ETA: 0s - loss: 0.5773
Epoch 37: val_loss did not improve from 0.59239
93/93 [==============================] - 1s 8ms/step - loss: 0.5730 - val_loss: 0.6150
Epoch 38/100
87/93 [===========================>..] - ETA: 0s - loss: 0.5581
Epoch 38: val_loss did not improve from 0.59239
93/93 [==============================] - 1s 8ms/step - loss: 0.5608 - val_loss: 0.6162
Epoch 39/100
91/93 [============================>.] - ETA: 0s - loss: 0.5705
Epoch 39: val_loss did not improve from 0.59239
93/93 [==============================] - 1s 8ms/step - loss: 0.5699 - val_loss: 0.6284
Epoch 40/100
87/93 [===========================>..] - ETA: 0s - l

89/93 [===========================>..] - ETA: 0s - loss: 0.5316
Epoch 73: val_loss did not improve from 0.58269
93/93 [==============================] - 1s 8ms/step - loss: 0.5319 - val_loss: 0.6512
Epoch 74/100
87/93 [===========================>..] - ETA: 0s - loss: 0.5435
Epoch 74: val_loss did not improve from 0.58269
93/93 [==============================] - 1s 8ms/step - loss: 0.5398 - val_loss: 0.6051
Epoch 75/100
88/93 [===========================>..] - ETA: 0s - loss: 0.5391
Epoch 75: val_loss did not improve from 0.58269
93/93 [==============================] - 1s 8ms/step - loss: 0.5420 - val_loss: 0.6117
Epoch 76/100
87/93 [===========================>..] - ETA: 0s - loss: 0.5273
Epoch 76: val_loss did not improve from 0.58269
93/93 [==============================] - 1s 8ms/step - loss: 0.5270 - val_loss: 0.6323
Epoch 77/100
88/93 [===========================>..] - ETA: 0s - loss: 0.5388
Epoch 77: val_loss did not improve from 0.58269
93/93 [==============================] -

In [23]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.682449,0.262755,0.669025,0.879952,0.262755,0.179035
